# 8 向量数据库
存储和搜索非结构化数据的最常见方法之一是嵌入它并存储生成的嵌入向量，

然后在查询时嵌入非结构化查询并检索与嵌入查询“最相似”的嵌入向量。

矢量存储负责存储嵌入数据并为您执行矢量搜索。

<img src="https://python.langchain.com/assets/images/vector_stores-9dc1ecb68c4cb446df110764c9cc07e0.jpg" alt="图片" width="900" height="500">

有许多很棒的矢量存储选项，这里有一些是免费的、开源的，并且完全在本地计算机上运行。

比如chrome，faiss，lance


 我们先体验一下chrome

In [1]:
! pip install chromadb


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 构建数据集
text = """客户：唉你好。\n客服：您好，建行1917为您服务，嗯。\n客户：唉，你好，我这边有我那个在咱银行那边有一张信用卡，然后上一次做了一个协商处理，现在也没有回复，我，想问一下我那个工单情况怎么样了？\n客服：呃你说那个信用卡的一个工单，我这边给您转接信用卡坐席那边可以吗？\n客户：唉，好的，谢谢你。\n客服：好不客气的，稍等一下。\n客户：嗯。\n客户：行好的。\n客服：如果转接不成功的话请重新拨打，选择3号键信用卡业务，我现在给您转接，您不要挂机，稍等一下。\n客户：唉，好的谢谢。\n客服：您好，您好，请问有什么可以帮您您好。\n客户：唉，你好，我这边前两天给咱。\n客户：那个下了工单，我想问一下信用卡账单那个信用卡上信用卡工单情况了。\n客服：信用卡工单情况之前反馈的信用卡的相关问题是吗？\n客户：嗯知道。\n客服：嗯，卡号或身份证号能告诉我一下吗？查一下记录。\n客户：941951。\n客服：嗯。\n客户：1115。\n客服：嗯。\n客户：1110。\n客服：嗯。\n客户：2217。\n客服：给您查一下，稍等啊。\n客户：嗯。\n客户：唉。\n客服：不好意思没有查到记录啊，我确认一下那个身份证号941911。\n客户：941951。\n客户：对二八。\n客服：二八，噢，不好意思，嗯，111511102217。\n客户：啊，对是的。\n客服：嗯。\n客服：呃是一个协商还款的问题。\n客户：对，是的，是的是的。\n客服：嗯嗯，现在的话是看一下啊。\n客服：有接到回复了吗？因为我看到4月6号显示有跟您做过联系。\n客户：然后他说怎么调取两年内的这个记录，然后因为我的逾期表长了五年了，然后所以他也他要他说要给我。\n客户：这几天给我回复了，一直没再回复啊。\n客服：一直没再回复明白了，这样吧，我们这边的话再给您就是催一下，让他们尽快给您回一下好吗？\n客户：行好的你再帮我下工单，然后让他给我回个电话啊。\n客服：唉，行行行，没问题，我给您去催单，嗯。\n客户：嗯。"""
list_text = text.split('\n')

list_text

['客户：唉你好。',
 '客服：您好，建行1917为您服务，嗯。',
 '客户：唉，你好，我这边有我那个在咱银行那边有一张信用卡，然后上一次做了一个协商处理，现在也没有回复，我，想问一下我那个工单情况怎么样了？',
 '客服：呃你说那个信用卡的一个工单，我这边给您转接信用卡坐席那边可以吗？',
 '客户：唉，好的，谢谢你。',
 '客服：好不客气的，稍等一下。',
 '客户：嗯。',
 '客户：行好的。',
 '客服：如果转接不成功的话请重新拨打，选择3号键信用卡业务，我现在给您转接，您不要挂机，稍等一下。',
 '客户：唉，好的谢谢。',
 '客服：您好，您好，请问有什么可以帮您您好。',
 '客户：唉，你好，我这边前两天给咱。',
 '客户：那个下了工单，我想问一下信用卡账单那个信用卡上信用卡工单情况了。',
 '客服：信用卡工单情况之前反馈的信用卡的相关问题是吗？',
 '客户：嗯知道。',
 '客服：嗯，卡号或身份证号能告诉我一下吗？查一下记录。',
 '客户：941951。',
 '客服：嗯。',
 '客户：1115。',
 '客服：嗯。',
 '客户：1110。',
 '客服：嗯。',
 '客户：2217。',
 '客服：给您查一下，稍等啊。',
 '客户：嗯。',
 '客户：唉。',
 '客服：不好意思没有查到记录啊，我确认一下那个身份证号941911。',
 '客户：941951。',
 '客户：对二八。',
 '客服：二八，噢，不好意思，嗯，111511102217。',
 '客户：啊，对是的。',
 '客服：嗯。',
 '客服：呃是一个协商还款的问题。',
 '客户：对，是的，是的是的。',
 '客服：嗯嗯，现在的话是看一下啊。',
 '客服：有接到回复了吗？因为我看到4月6号显示有跟您做过联系。',
 '客户：然后他说怎么调取两年内的这个记录，然后因为我的逾期表长了五年了，然后所以他也他要他说要给我。',
 '客户：这几天给我回复了，一直没再回复啊。',
 '客服：一直没再回复明白了，这样吧，我们这边的话再给您就是催一下，让他们尽快给您回一下好吗？',
 '客户：行好的你再帮我下工单，然后让他给我回个电话啊。',
 '客服：唉，行行行，没问题，我给您去催单，嗯。',
 '客户：嗯。']

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
# 使用向量数据库chroma，利用openai的embadding对文本进行向量化
db = Chroma.from_texts(list_text, OpenAIEmbeddings())

In [5]:
# 相似性搜索
query = "客户的身份证号是多少"
docs = db.similarity_search(query)
print(docs[0].page_content)

客服：嗯，卡号或身份证号能告诉我一下吗？查一下记录。


In [6]:
# 按向量搜索相似性
embedding_vector = OpenAIEmbeddings().embed_query(query)
print(f'embedding_vector：{embedding_vector}')
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

embedding_vector：[-0.0006600301256486075, -0.003293202962316684, 0.013228393176677565, -0.05032903353152666, -0.04157495211258094, 0.009740655568422981, -0.04037994984795809, -0.014437290773153123, -0.041936230740751326, -0.013436824085790682, 0.0304586531028049, 0.018327994867781233, -0.02708207814937197, 0.0023153161892148916, -0.0017681859841154706, 0.03240400701276397, 0.00449862626016781, -0.036517036417035395, 0.05541473989077762, -0.0021798364708203698, -0.010678592331125732, 0.029041325528538733, -0.039240528910746066, 0.015104268564728084, -0.024080679018607155, 0.0011394204715507678, 0.005391404125671772, -0.005471302283824838, 0.021301603334840633, -0.04404832863294289, 0.0177165984036171, -0.01699403928463131, -0.028680046900368347, -0.02092642937481754, -0.035155290170180056, -0.001811609012570494, 0.007843937182593401, -0.006711464376968988, 0.02506724944279438, 0.013652202660830151, 0.01935625128752658, 0.014437290773153123, -0.0014841993244685727, 0.02502556344723626, -

我们看一下faiss

如果你的电脑显卡比较好可以pip install faiss-gpu


In [7]:
! pip install faiss-cpu


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
db = FAISS.from_texts(list_text, OpenAIEmbeddings())

In [10]:
# 相似性搜索
query = "客户的身份证号是多少"
docs = db.similarity_search(query)
print(docs[0].page_content)

客服：嗯，卡号或身份证号能告诉我一下吗？查一下记录。


In [11]:
# 按向量搜索相似性
embedding_vector = OpenAIEmbeddings().embed_query(query)
print(f'embedding_vector：{embedding_vector}')
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

embedding_vector：[-0.0006290878419341604, -0.003268823907417372, 0.013221272059036733, -0.05041044498510392, -0.04165187039893983, 0.0097525993687725, -0.04042845155659308, -0.014444690901383487, -0.04204114087725249, -0.013395053123430931, 0.030418656287022977, 0.018309585461547755, -0.02709596002612599, 0.0023599483166496086, -0.0016943664255812964, 0.032420616085995044, 0.004535688481523954, -0.036480143091347964, 0.05547095002729775, -0.002130557283709592, -0.010691017675294697, 0.029056213338218912, -0.039260643846945345, 0.015112010523933324, -0.024120827756662515, 0.0011278397691573533, 0.005411545660743595, -0.005522765597835234, 0.021298620514185986, -0.04407090624257407, 0.017739582527253526, -0.017002749920152473, -0.028694748426255766, -0.02090935003587333, -0.03517331127524292, -0.0019029038658357803, 0.007764542818636465, -0.0066975265617154655, 0.025024490036570386, 0.013714810617192373, 0.01933836906473818, 0.014472496467732996, -0.0014458594150219522, 0.024954976120696

In [12]:
#带分数的查找
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

[(Document(page_content='客服：嗯，卡号或身份证号能告诉我一下吗？查一下记录。', metadata={}),
  0.23871653),
 (Document(page_content='客服：不好意思没有查到记录啊，我确认一下那个身份证号941911。', metadata={}),
  0.29620746),
 (Document(page_content='客户：那个下了工单，我想问一下信用卡账单那个信用卡上信用卡工单情况了。', metadata={}),
  0.34041685),
 (Document(page_content='客服：呃你说那个信用卡的一个工单，我这边给您转接信用卡坐席那边可以吗？', metadata={}),
  0.344947)]

In [ ]:
# 保存和加载
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", OpenAIEmbeddings())